# Test Correspondences

In [ ]:
%matplotlib widget
import numpy as np
import os
from plotly.subplots import make_subplots

import planeslam.io as io
from planeslam.general import NED_to_ENU
from planeslam.scan import pc_to_scan
from planeslam.registration import extract_corresponding_features, get_correspondences

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

Read in airsim LiDAR and pose data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

In [ ]:
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [ ]:
# Convert to ENU
num_scans = len(PC_data)

for i in range(num_scans):
    PC_data[i] = NED_to_ENU(PC_data[i])

drone_positions = NED_to_ENU(drone_positions)
drone_orientations = NED_to_ENU(drone_orientations)

Extract planes 

In [ ]:
scans = num_scans * [None]

for i in range(num_scans):
    scans[i] = pc_to_scan(PC_data[i])

In [ ]:
# Plot 2 scans
source = scans[11]
target = scans[12]

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=1)

for t in target.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1200, height=500, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

correspondences = get_correspondences(source, target)
print(correspondences)

In [ ]:
from planeslam.registration import decoupled_GN_register

R_hat, t_hat = decoupled_GN_register(source, target)

print(R_hat)
print(t_hat)

In [ ]:
source.transform(R_hat, t_hat.flatten())

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=1)

for t in target.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1200, height=500, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()